In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.chdir('/content/gdrive/MyDrive/TPS')

In [ ]:
!pip install folium

In [314]:
import folium
import numpy as np
import matplotlib.pyplot as plt 
import random 

class random_generator(object):
    def __init__(self,num=20, range_=(0,100), start_point=0):
        self.points_x = np.random.uniform(126.734086, 127.269311,size=(num, 1))
        self.points_y = np.random.uniform(37.413294,37.715133, size=(num, 1))
        self.points = np.hstack((self.points_x, self.points_y))
        self.start_point = 0

    def visualize(self):
        m = create_maposm(self.points[:,0], self.points[:,1])
        make_marker(m,self.points[:,0], self.points[:,1])
        m


    def get_points(self):
        return self.points


def distance_matric(from_, to_):
    return np.sqrt((from_[0]-to_[0])**2 +(from_[1]-to_[1])**2)


class environment(object):
    def __init__(self, points_obj):
        self.points_obj = points_obj
        self.points = self.points_obj.get_points()
        self.current_position = self.points_obj.start_point
        self.state = -np.ones(len(self.points_obj.points)) # initailize with 0 , [0,0...] ,
        self.state[self.current_position]=1

        self.next_state = self.state
        self.history = [self.current_position] 
        self.reward_history=[]
       

    def is_done(self): 
        done_mask = False 
        if sum(self.state) == len(self.points_obj.points): # 다 돌았다. :
            done_mask = True 

        return done_mask

    def reward(self, action):
        distance = distance_matric(from_ =self.points[self.current_position], to_= self.points[action])
        return -distance 
    
    def reset(self, random_init =True):
        self.points_obj = self.points_obj
        self.points = self.points_obj.get_points()
        self.current_position = self.points_obj.start_point
        self.state = -np.ones(len(self.points_obj.points)) # initailize with 0 , [0,0...] ,

        if random_init == True :
            rand = np.random.randint(0,len(self.points_obj.points))
            self.state[rand]=1
        else :
            self.state[self.current_position]=1

        self.next_state = self.state
        self.history = [self.current_position] 
        self.reward_history=[]

        return self.state.tolist()

    def transition(self, action): 
        reward = self.reward(action)      
        self.current_position = action  
        self.state[self.current_position]=1 
        self.history.append(self.current_position)
        self.reward_history.append(reward)
        return [self.state.tolist(), reward,self.is_done()] 

    def visualize(self,save=0,save_name=None):
        m = create_maposm(self.points[:,0], self.points[:,1])
        make_marker(m,self.points[:,0], self.points[:,1])
        for i in range(len(self.history)-1):
            point_1 =self.points_obj.points[self.history[i]]
            point_2 = self.points_obj.points[self.history[i+1]]
            #draw_line(m, point_1, point_2)
            #folium.PolyLine(gen.points[self.history].tolist(), color="red", weight=2.5, opacity=1).add_to(map_1)
        if save >0 :
            m.save(save_name+"/image_"+str(save)+".png")
        return m 


    # def visualize_geo(self):
    #     point_1 =self.points_obj.points[self.history[i]]
    #     point_2 = self.points_obj.points[self.history[i+1]]

    #     map_1 = g.Map(location=[np.mean(points_obj.points_y), np.mean(points_obj.points_x[i])])
    #     for i in range(len(self.history)):
    #             g.Marker([points_obj.points_y[i][0],points_obj.points_x[i][0]],icon = g.Icon(color='green',icon='plus')).add_to(map_1)
    #     g.PolyLine(gen.points[history].tolist(), color="red", weight=2.5, opacity=1).add_to(map_1)

    #     return map_1




In [315]:
rand_gen.points[:,0]

array([127.03664649, 127.02997097, 126.74553628, 126.9210124 ,
       127.11308947, 126.93957164, 127.25029527, 126.81132404,
       126.98845629, 127.10381769, 127.22965687, 127.067438  ,
       126.78630944, 126.75286547, 127.19012983, 126.77293921,
       127.2227143 , 126.90118071, 126.91342792, 126.84661037])

In [316]:
gen.points_x

array([[126.8646264 ],
       [126.83273092],
       [127.26341193],
       [127.03547919],
       [127.10929857],
       [126.82490247],
       [126.85301881],
       [127.05729467],
       [126.88465964],
       [127.09539287]])

In [317]:
env.state

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [318]:

rand_gen=random_generator()



In [319]:





len(rand_gen.points)

20

In [320]:
env=environment(rand_gen)

In [324]:
client_id = '8udacwedaf';    
client_pw = 'dobOVjHUa8MUvGkMaTEEDhwHPB2hcwuLl7SgC1yY';
api_url = 'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords='


def reverse_geocoding(cord_dataset):
    results=[]
    for cord in cord_dataset:
        cord_url = parse.quote(cord[0])
        url = api_url + cord_url + "&orders=roadaddr&output=json"
        request = Request(url)
        request.add_header('X-NCP-APIGW-API-KEY-ID',client_id)
        request.add_header('X-NCP-APIGW-API-KEY', client_pw)
        try:
            response = urlopen(request)
        except HTTPError as e:
            print('HTTP Error!', e)
        else: 
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read().decode('utf-8')
                response_body = json.loads(response_body)
                print(response_body)
                if response_body['status']['code'] == 0:
                  area1 = response_body['results'][0]['region']['area1']['name']
                  area2 = response_body['results'][0]['region']['area2']['name']
                  name = response_body['results'][0]['land']['name']
                  land1 = response_body['results'][0]['land']['number1']
                  addition = response_body['results'][0]['land']['addition0']['value']
                  print(area1, area2, name, land1, addition)
                  results.append(area1+ " " + area2 + " " + name + " " + land1 + " " + addition)
                else: 
                  results.append("주소 정보가 없습니다")
    return results
                # response_body = json.loads(response_body) 
                # print(response_body['region'][0])
                # print("Success!")



In [325]:
from urllib import request
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
import xml.etree.ElementTree as ET

In [326]:
import folium
import numpy as np

def create_maposm(xs, ys):
    x_c = np.median(xs)
    y_c = np.median(ys)
    map_osm = folium.Map(location=[y_c,x_c])
    return map_osm
def make_marker(map_osm, x, y):
    data = []
    for i in range(len(x)):
        data.append([str(x[i])+ ", "+str(y[i])])
        print(data)
    geoname = reverse_geocoding(data)
    for i in range(len(x)):
        folium.Marker([y[i],x[i]], tooltip=geoname[i]).add_to(map_osm)
    return map_osm

def draw_line(map_osm, point1, point2):
    location = [point1, point2]
    folium.PolyLine(locations=location, tooltip='Polyline').add_to(map_osm)
    return map_psm

In [327]:
rand_gen.points

array([[126.84752436,  37.48386816],
       [126.86062269,  37.4183489 ],
       [127.0033503 ,  37.57514967],
       [127.03442559,  37.52483111],
       [127.15886926,  37.66335518],
       [126.99855384,  37.70082079],
       [127.24273036,  37.65338494],
       [127.19952469,  37.68536923],
       [126.91336483,  37.47624457],
       [127.00977254,  37.58098636],
       [126.78192916,  37.70946788],
       [127.06248563,  37.68231575],
       [127.23383861,  37.55158366],
       [126.89362945,  37.46567569],
       [126.92546645,  37.45976228],
       [127.1102856 ,  37.55835454],
       [127.16269857,  37.6197655 ],
       [127.07857166,  37.64279828],
       [127.06189975,  37.47660915],
       [127.13176104,  37.51110766]])

In [328]:

s, r, done = env.transition(2)
print(s,r,done)

[1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0] -0.18059357078415642 False


In [329]:
env.reset()

[-1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0]

In [330]:
import numpy as np 

rand_generator = random_generator(num=20)
rand_generator.visualize()
env = environment(points_obj=rand_generator)
done = False 
i = 0 
state =env.reset()
while not done : 
    #random_mover_action = np.random.randint(0,20)
 
    possible_node = (np.where(np.array(state)==-1)[0])
    random_mover_action=random.choice(possible_node)
    state,reward,done,=env.transition(random_mover_action)
    print(state,done)
    possible_node = (np.where(np.array(state)==-1)[0])
    i+=1
    if done :
        break
    else:pass


[['126.93229461907063, 37.59847019865914']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126.82631220977417, 37.63959005093059']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126.82631220977417, 37.63959005093059'], ['126.8908130686419, 37.455163461334905']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126.82631220977417, 37.63959005093059'], ['126.8908130686419, 37.455163461334905'], ['126.8758196509127, 37.54983501776668']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126.82631220977417, 37.63959005093059'], ['126.8908130686419, 37.455163461334905'], ['126.8758196509127, 37.54983501776668'], ['127.13306988563984, 37.584807471972844']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126

In [331]:
env.visualize()

[['126.93229461907063, 37.59847019865914']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126.82631220977417, 37.63959005093059']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126.82631220977417, 37.63959005093059'], ['126.8908130686419, 37.455163461334905']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126.82631220977417, 37.63959005093059'], ['126.8908130686419, 37.455163461334905'], ['126.8758196509127, 37.54983501776668']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126.82631220977417, 37.63959005093059'], ['126.8908130686419, 37.455163461334905'], ['126.8758196509127, 37.54983501776668'], ['127.13306988563984, 37.584807471972844']]
[['126.93229461907063, 37.59847019865914'], ['127.04620571811007, 37.59786146300367'], ['126

In [67]:
'image_3.png'.split("_")[1][0]

'3'